In [1]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment=None

In [2]:
pd.set_option('display.max_columns',60)


In [3]:
import matplotlib.pyplot as plt


In [4]:
%matplotlib inline

In [5]:
plt.rcParams['font.size']=24

In [6]:
from IPython.core.pylabtools import figsize

In [8]:
import seaborn as sns
sns.set(font_scale=2)
from sklearn.preprocessing import Imputer,MinMaxScaler

In [13]:
from sklearn.linear_model import LinearRegression


In [15]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor


In [16]:
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor

In [17]:
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV

In [20]:
train_features=pd.read_csv('data/training_features.csv')
test_features=pd.read_csv('data/testing_features.csv')
train_labels=pd.read_csv('data/testing_labels.csv')
test_labels=pd.read_csv('data/testing_labels.csv')
